In [44]:
import pymongo
import random
import string
from datetime import datetime, timedelta

In [45]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
homecare_db = client["homecare"]
eportal_db = client["eportal"]
s2travel_db = client["s2travel"]

In [46]:
homecare_users = homecare_db["users"]
eportal_users = eportal_db["users"]
s2travel_users = s2travel_db["users"]

In [50]:
def get_random_email():
    validchars='abcdefghijklmnopqrstuvwxyz1234567890'
    loginlen=random.randint(4,15)
    login=''
    for i in range(loginlen):
        pos=random.randint(0,len(validchars)-1)
        login=login+validchars[pos]
    if login[0].isnumeric():
        pos=random.randint(0,len(validchars)-10)
        login=validchars[pos]+login
    servers=['@gmail','@yahoo','@redmail','@hotmail','@bing']
    servpos=random.randint(0,len(servers)-1)
    email=login+servers[servpos]
    tlds=['.com','.in','.gov','.ac.in','.net','.org']
    tldpos=random.randint(0,len(tlds)-1)
    email=email+tlds[tldpos]
    return email

def get_random_name():
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(7))

def get_random_address():
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(30))

for i in range(50):
    fname = get_random_name().capitalize()
    lname = get_random_name().capitalize()
    dob = datetime.now() - timedelta(days=random.randint(365*18, 365*80))
    location = {"lat": random.uniform(-90, 90), "long": random.uniform(-180, 180)}
    
    homecare1_data = {"fname": fname, "lname": lname, "dob": dob, "location": location, "mail":get_random_email()}
    eportal_data = {"firstname": fname, "lastname": lname, "date_of_birth": dob, "location": location ,"email":get_random_email()}
    s2travel_data = {"first_name": fname, "last_name": lname, "DOB": dob, "loc": location,"EMAIL":get_random_email(), "address":get_random_address()}
    
    homecare_users.insert_one(homecare1_data)
    eportal_users.insert_one(eportal_data)
    s2travel_users.insert_one(s2travel_data)
    
print("done")


done


In [64]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")

dbs = [
    {"name": "homecare", "fields": ["fname", "lname", "dob", "location","mail"]},
    {"name": "eportal", "fields": ["firstname", "lastname", "date_of_birth", "location","email"]},
    {"name": "s2travel", "fields": ["first_name", "last_name", "DOB", "location","EMAIL", "address"]}
]

db4_name = "profiler"
collection_name = "users"

db4 = client[db4_name]
users = db4[collection_name]

users.create_index([("EMAIL", pymongo.ASCENDING)], unique=True)

for db in dbs:
    database = client[db["name"]]
    users_collection = database["users"]
    
    for user_data in users_collection.find():
        keys = ["FIRST_NAME", "LAST_NAME", "DOB","LOCATION","EMAIL","ADDRESS"]
        
        combined_user_data = {}
        for key in keys:
            if key == "FIRST_NAME":
                combined_user_data[key] = user_data.get(db["fields"][0], "").capitalize()
            elif key == "LAST_NAME":
                combined_user_data[key] = user_data.get(db["fields"][1], "").capitalize()
            elif key == "DOB":
                combined_user_data[key] = user_data.get(db["fields"][2], "")
            elif key == "LOCATION":
                combined_user_data[key] = user_data.get(db["fields"][3], "")
            elif key == "EMAIL":
                combined_user_data[key] = user_data.get(db["fields"][4], "")
            elif key == "ADDRESS" and db['name'] == 's2travel':
                combined_user_data[key] = user_data.get(db["fields"][5], "")
                
        existing_user = users.find_one({"EMAIL": combined_user_data["EMAIL"]})
        if existing_user is not None:
            print(f"Skipping user with email {combined_user_data['EMAIL']} as it already exists")
            continue
                
                
        try:
            users.insert_one(combined_user_data)
        except e:
            print(e)
    
        

    
print(f"All user data inserted into {db4_name}.{collection_name} collection")

All user data inserted into profiler.users collection
